# GUI Review Hits

Review initial hits identified to find the real positives and the false positives

## Configuration

In [1]:
#subdir = 'dashcam_tour_frankston'
#suffix = ''

subdir = 'dashcam_tour_mount_eliza'
suffix = '_V5_mask2'

max_range_from_hit = 50

## Code

In [2]:
import os
import sys
import ipywidgets as widgets
import pandas as pd

from pathlib import Path
from shutil import copyfile

from geopy.distance import geodesic

from tqdm.notebook import tqdm

module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)

In [3]:
detections_dir     = os.path.join(os.pardir, 'data_sources', subdir, 'detections'+ suffix)
hits_dir           = os.path.join(detections_dir, 'hits')
miss_dir           = os.path.join(detections_dir, 'miss')
split_dir          = os.path.join(os.pardir, 'data_sources', subdir, 'split')
true_positive_dir  = os.path.join(hits_dir, 'true_positives')
false_positive_dir = os.path.join(hits_dir, 'false_positives')
near_miss_dir      = os.path.join(miss_dir, 'near_miss')
far_miss_dir       = os.path.join(miss_dir, 'far_miss')

detection_log     = os.path.join(detections_dir, 'detection_log.csv')

Path(true_positive_dir).mkdir(parents=True, exist_ok=True)
Path(false_positive_dir).mkdir(parents=True, exist_ok=True)
Path(near_miss_dir).mkdir(parents=True, exist_ok=True)
Path(far_miss_dir).mkdir(parents=True, exist_ok=True)

In [4]:
df = pd.read_csv(detection_log)

df.head(5)

global image_index
image_index = 0

In [5]:
def fetch_image_filenames(lat, lon, bearing, path):
    abs_lat = abs(lat)
    abs_lon = abs(lon)
    
    if lat < 0:
        str_lat = 's'
    else:
        str_lat = 'n'
        
    if lon < 0:
        str_lon = 'e'
    else:
        str_lon = 'w'
        
    bbox_filename = '{0:s}{1:.6f}_{2:s}{3:.6f}_{4:d}.jpg'.format(
        str_lat,
        abs_lat,
        str_lon,
        abs_lon,
        int(bearing)
    )
    
    orig_filename = os.path.basename(path)
    
    return orig_filename, bbox_filename

## GUI to review "Hits" for True Positive vs. False Positive

In [6]:
# Set up callback functions
def update_image(index):
    if index >= len(df):
        print('No more images to process')
        return
    
    # Get details of next detection
    row = df.iloc[[index]]
       
    lat       = row['lat'].item()
    lon       = row['lon'].item()
    bearing   = row['bearing'].item()
    orig_path = row['orig_filename'].item()

    global orig_filename
    global bbox_filename
    
    orig_filename, bbox_filename = fetch_image_filenames(lat, lon, bearing, orig_path)
    
    filename = os.path.join(hits_dir, bbox_filename)
    file     = open(filename, 'rb')
    image    = file.read()
    
    image_widget.value = image
    count_widget.value = '{0:d} of {1:d}'.format(index+1, len(df))

# Create widgets
image_widget = widgets.Image(format='jpg')
count_widget = widgets.Text(value='{0:d} of {1:d}'.format(1, len(df)))
hit_button   = widgets.Button(description='True Postiive')
miss_button  = widgets.Button(description='False Positive')
dup_button   = widgets.Button(description='Duplicate')
row          = widgets.HBox([hit_button, miss_button, dup_button, count_widget])
out          = widgets.Output()

@out.capture()

def next_image():
    global image_index
    image_index += 1
    update_image(image_index)
    
def on_hit(b):
    copyfile(os.path.join(split_dir,orig_filename), os.path.join(true_positive_dir, orig_filename))
    next_image()
    
def on_miss(b):
    copyfile(os.path.join(split_dir,orig_filename), os.path.join(false_positive_dir, orig_filename))
    next_image()
    
def on_duplicate(b):
    next_image()

hit_button.on_click(on_hit)
miss_button.on_click(on_miss)

# Start with the first image
update_image(0)

# Display the GUI
display(image_widget)
display(row)
display(out)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Output()

## Find Misses that were not within range of a hit

In [7]:
global hit_coords
hit_coords = []

for index, row in df.iterrows():
    hit_coords.append((row['lat'], row['lon']))
    
def find_closest_hit_distance(lat, lon):
    global hit_coords
    
    min_distance = None
    
    for idx, coords in enumerate(hit_coords):
        distance = geodesic((lat, lon), coords).m
        
        if min_distance is None or distance < min_distance:
            min_distance = distance
            
    return min_distance


In [9]:
miss_files = [f for f in os.listdir(miss_dir) if f.endswith('.jpg')]

for i in tqdm(range(len(miss_files))):
    miss_file = miss_files[i]
    
    # Parse the filename back to co-ordinates
    
    filename_parts = miss_file.split('_')
    lat_str = filename_parts[0]
    lon_str = filename_parts[1]
    
    if lat_str.startswith('s'):
        lat = -float(lat_str[1:])
    else:
        lat = float(lat_str[1:])
        
    if lon_str.startswith('e'):
        lon = -float(lon_str[1:])
    else:
        lon = float(lon_str[1:])
    
    min_distance = find_closest_hit_distance(lat, lon)
    
    if min_distance <= max_range_from_hit:
        copyfile(os.path.join(miss_dir, miss_file), os.path.join(near_miss_dir, miss_file))
    else:
        copyfile(os.path.join(miss_dir, miss_file), os.path.join(far_miss_dir, miss_file))    

  0%|          | 0/12417 [00:00<?, ?it/s]